6. The SHAP analysis is interesting, however it could be made stronger by looking into the correlations of features and/or the stability of SHAP scores for the same MTL model across different folds/seeds. Basically, are the differences in SHAP scores statistically significant?


In [1]:
import argparse
from datetime import datetime
from pathlib import Path
from warnings import filterwarnings

import lightning.pytorch as pl
import pandas as pd
import torch
import yaml
from tqdm import tqdm

from dataset import TCGA_Program_Dataset
from datasets_manager import TCGA_Balanced_Datasets_Manager, TCGA_Datasets_Manager
from lit_models import LitFullModel
from model import Classifier, Feature_Extractor, Graph_And_Clinical_Feature_Extractor, Task_Classifier
from utils import config_add_subdict_key, get_logger, override_n_genes, set_random_seed, setup_logging
import shap


import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('TkAgg')
import numpy as np
import seaborn as sns

SEED = 1126
set_random_seed(SEED)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:


def create_models_and_optimizers(config: dict):
    models: dict[str, torch.nn.Module] = {}
    optimizers: dict[str, torch.optim.Optimizer] = {}

    # Setup models. Do not use getattr() for better IDE support.
    for model_name, kargs in config['models'].items():
        if model_name == 'Graph_And_Clinical_Feature_Extractor':
            models['feat_ext'] = Graph_And_Clinical_Feature_Extractor(**kargs)
        elif model_name == 'Feature_Extractor':
            models['feat_ext'] = Feature_Extractor(**kargs)
        elif model_name == 'Task_Classifier':
            models['clf'] = Task_Classifier(**kargs)
        elif model_name == 'Classifier':
            models['clf'] = Classifier(**kargs)
        else:
            raise ValueError(f'Unknown model type: {model_name}')

    # Setup optimizers. If the key is 'all', the optimizer will be applied to all models.
    for key, optim_dict in config['optimizers'].items():
        opt_name = next(iter(optim_dict))
        if key == 'all':
            params = [param for model in models.values() for param in model.parameters()]
            optimizers[key] = getattr(torch.optim, opt_name)(params, **optim_dict[opt_name])
        else:
            optimizers[key] = getattr(torch.optim, opt_name)(models[key].parameters(), **optim_dict[opt_name])

    # Add models' structure to config for logging. TODO: Prettify.
    for model_name, torch_model in models.items():
        config[f'model.{model_name}'] = str(torch_model)
    return models, optimizers

In [3]:

# Select a config file.

config_path = '/home/zow/Multi-Cancer/config/light/MTL_TCGA.yaml'
config = config_path
with open(config, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
override_n_genes(config)                                                    # For multi-task graph models.
config_name =config_path.split('/')[-1].split('.')[0]


In [4]:

# Setup logging.
setup_logging(log_path := f'Logs/{config_name}/{datetime.now():%Y-%m-%dT%H:%M:%S}/')
logger = get_logger(config_name)
logger.info(f'Using Random Seed {SEED} for this experiment')
get_logger('lightning.pytorch.accelerators.cuda', log_level='WARNING')      # Disable cuda logging.
filterwarnings('ignore', r'.*Skipping val loop.*')                          # Disable val loop warning.


[INFO]	Using Random Seed 1126 for this experiment


In [5]:

# Create dataset manager.
#here use torch lightning DS
data = {'TCGA_BLC': TCGA_Program_Dataset(**config['datasets'])}
if 'TCGA_Balanced_Datasets_Manager' == config['datasets_manager']['type']:
    manager = TCGA_Balanced_Datasets_Manager(datasets=data, config=config_add_subdict_key(config))
else:
    manager = TCGA_Datasets_Manager(datasets=data, config=config_add_subdict_key(config))


[INFO]	Creating a TCGA Program Dataset with 3 Projects...
[INFO]	All files are downloaded for TCGA-BRCA
[INFO]	Creating 1093 cases for TCGA-BRCA...
[INFO]	Using genomic tpm cache files created at 2024-02-23 10:53:54 for TCGA-BRCA
[INFO]	Using clinical cache files created at 2024-02-23 10:54:21 for TCGA-BRCA
[INFO]	Using vital status cache files created at 2024-02-23 10:54:29 for TCGA-BRCA
[INFO]	Using overall survival cache files created at 2024-02-23 10:54:37 for TCGA-BRCA
[INFO]	Using disease specific survival cache files created at 2024-02-23 10:54:46 for TCGA-BRCA
[INFO]	Using survival time cache files created at 2024-02-23 10:54:53 for TCGA-BRCA
[INFO]	Using primary site cache files created at 2024-02-23 10:55:06 for TCGA-BRCA
[INFO]	All files are downloaded for TCGA-LUAD
[INFO]	Creating 510 cases for TCGA-LUAD...
[INFO]	Using genomic tpm cache files created at 2024-02-23 11:41:50 for TCGA-LUAD
[INFO]	Using clinical cache files created at 2024-02-23 11:41:58 for TCGA-LUAD
[INFO]	U

In [ ]:

valid_results = []
# Cross validation.
all_shap_values_folds = []
all_project_ids = []
for key, values in manager['TCGA_BLC']['dataloaders'].items():
    if isinstance(key, int) and config['cross_validation']:
        pass
        models, optimizers = create_models_and_optimizers(config)
        lit_model = LitFullModel(models, optimizers, config)
        trainer = pl.Trainer(                                               # Create sub-folders for each fold.
            default_root_dir="",#log_path,
            max_epochs=config['max_epochs'],
            log_every_n_steps=1,
            enable_model_summary=False,
            enable_checkpointing=False,
        )
        print(f'Fold {key}: saved in {log_path}')
        trainer.fit(lit_model, train_dataloaders=values['train'], val_dataloaders=values['valid'])

        
        # get one batch of data
        batch = next(iter(train))
        (genomic, clinical, index, project_id), (overall_survival, survival_time, vital_status) = batch
        # get the shap values
        feat_extractor = models['feat_ext']
        genomic_feat_ext = feat_extractor.genomic_feature_extractor        
        explainer = shap.DeepExplainer(genomic_feat_ext, genomic)
        #separate according to project_id
        project_id = project_id.numpy()
        all_project_ids.append(project_id)


        shap_values = explainer.shap_values(genomic)
        all_shap_values_folds.append(np.array(shap_values))
        #print shape of shap values and project_id
        print(np.array(shap_values).shape)
        print(project_id.shape)
        
        
    elif key == 'train':
        train = values
    elif key == 'test':
        test = values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Fold 0: saved in Logs/MTL_TCGA/2024-03-11T13:43:36/
Epoch 49: 100%|██████████| 10/10 [00:00<00:00, 24.75it/s, v_num=28, loss=0.406]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 10/10 [00:00<00:00, 24.66it/s, v_num=28, loss=0.406]


Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

Fold 2: saved in Logs/MTL_TCGA/2024-03-11T13:43:36/
Epoch 49: 100%|██████████| 10/10 [00:00<00:00, 24.43it/s, v_num=30, loss=0.384]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 10/10 [00:00<00:00, 24.34it/s, v_num=30, loss=0.384]


Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Fold 3: saved in Logs/MTL_TCGA/2024-03-11T13:43:36/
Epoch 42:  90%|█████████ | 9/10 [00:00<00:00, 39.84it/s, v_num=31, loss=0.395] 

In [ ]:
len(all_shap_values_folds[0])
all_shap_values_folds[0].shape


#aggregate shap values




4


In [ ]:
all_shap_values_folds_list = []
for fold in range(len(all_shap_values_folds)):
    shap_values = all_shap_values_folds[fold]
    # average shap values collapsing the 1s dimension
    shap_values = np.mean(shap_values, axis = 0)
    print(shap_values.shape)
    project_id = all_project_ids[fold]
    # save shap values in different dfs, according to project id
    for i in range(len(np.unique(project_id))):
        # add project id as another column
        shap_values_df = pd.DataFrame(shap_values)
        shap_values_df['project_id'] = project_id
        #shap_values_df.to_csv(f'shap_values_fold{fold}_project{i}.csv')
    shap_values_df["Fold"] = fold
    all_shap_values_folds_list.append(shap_values_df)

#concatenate all dfs
all_shap_values_folds_df = pd.concat(all_shap_values_folds_list)
all_shap_values_folds_df.to_csv('shap_values_all_folds.csv')
all_shap_values_folds_df.head()


In [ ]:
# separate according to project_id
BRCA_shap_values = all_shap_values_folds_df[all_shap_values_folds_df['project_id'] == 0]
COAD_shap_values = all_shap_values_folds_df[all_shap_values_folds_df['project_id'] == 2]
LUAD_shap_values = all_shap_values_folds_df[all_shap_values_folds_df['project_id'] == 1]

#drop project_id column
BRCA_shap_values = BRCA_shap_values.drop(columns = ['project_id'])
COAD_shap_values = COAD_shap_values.drop(columns = ['project_id'])
LUAD_shap_values = LUAD_shap_values.drop(columns = ['project_id'])

#relabel gene names

BRCA_genes = ['ESR1', 'EFTUD2', 'HSPA8', 'STAU1', 'SHMT2', 'ACTB', 'GSK3B', 'YWHAB', 'UBXN6', 'PRKRA', 'BTRC', 'DDX23', 'SSR1', 'TUBA1C', 'SNIP1', 'SRSF5', 'ERBB2', 'MKI67', 'PGR', 'PLAU']
LUAD_genes = ['HNRNPU', 'STAU1', 'KDM1A', 'SERBP1', 'DHX9', 'EMC1', 'SSR1', 'PUM1', 'CLTC', 'PRKRA', 'KRR1', 'OCIAD1', 'CDC73', 'SLC2A1', 'HIF1A', 'PKM', 'CADM1', 'EPCAM', 'ALCAM', 'PTK7']
COAD_genes =  ['HNRNPL', 'HNRNPU', 'HNRNPA1', 'ZBTB2', 'SERBP1', 'RPL4', 'HNRNPK', 'HNRNPR', 'TFCP2', 'DHX9', 'RNF4', 'PUM1', 'ABCC1', 'CD44', 'ALCAM', 'ABCG2', 'ALDH1A1', 'ABCB1', 'EPCAM', 'PROM1']
BRCA_shap_values.columns = BRCA_genes + ['Fold']
COAD_shap_values.columns = COAD_genes + ['Fold']
LUAD_shap_values.columns = LUAD_genes + ['Fold']


# plot SHAP values for BRCA
BRCA_shap_values = BRCA_shap_values.drop(columns = ['Fold'])
BRCA_shap_values = BRCA_shap_values.T
BRCA_shap_values['mean'] = BRCA_shap_values.mean(axis = 1)
BRCA_shap_values = BRCA_shap_values.sort_values(by = 'mean', ascending = False)
BRCA_shap_values = BRCA_shap_values.drop(columns = ['mean'])

plt.figure(figsize=(10, 10))
sns.heatmap(BRCA_shap_values, cmap = 'coolwarm')
plt.title('BRCA SHAP values')
plt.show()


#calculate stability accross folds
# Calculate the mean and standard deviation of SHAP values for each feature across folds
def calculate_stability(shap_df):
    # separate according to fold and save in a list

    means = shap_df.groupby('Fold').mean()
    stds = shap_df.groupby('Fold').std()
    cv = stds / means  # Coefficient of Variation
    columns_names = shap_df.columns[:-1]

    # Convert to DataFrame for easier handling and interpretation
    stability_df = pd.DataFrame({
        'mean_shap': means.mean(axis=0),  # Mean across samples, then across folds
        'std_shap': stds.mean(axis=0),    # Std across samples, then across folds
        'cv_shap': cv.mean(axis=0)        # CV across samples, then across folds
    }, index=columns_names)  # Assuming feature names are the same across all folds

    # Sort by CV for features
    stability_df = stability_df.sort_values(by='cv_shap')
    return stability_df



# Plot the Coefficient of Variation (CV) for each feature
def plot_SHAP_stability(stability_df, dataset_name = ''):
    plt.figure(figsize=(10, 8))
    sns.barplot(x='cv_shap', y=stability_df.index, data=stability_df)
    plt.title('Coefficient of Variation of SHAP Values Across Folds for ' + dataset_name)
    plt.xlabel('Coefficient of Variation (CV)')
    plt.ylabel('Features')
    #plt.show()



BRCA_stability = calculate_stability(BRCA_shap_values)
COAD_stability = calculate_stability(COAD_shap_values)
LUAD_stability = calculate_stability(LUAD_shap_values)


plot_SHAP_stability(BRCA_stability, 'BRCA')
plot_SHAP_stability(COAD_stability, 'COAD')
plot_SHAP_stability(LUAD_stability, 'LUAD')
plt.show()


Features with a high coefficient of variation (CV) are less stable across folds, meaning their importance varies more between different subsets of the data. In contrast, features with a low CV are more stable, suggesting consistent importance across different folds.

By identifying which features are stable and which are not, you can better understand the reliability of your model's feature importance scores and potentially identify areas where your model's performance might be improved with more consistent or representative training data.

In [ ]:
# Initialize a list to store all averaged SHAP values from all folds
averaged_shap_values_all_folds = []

for fold_shap_values in all_shap_values_folds:  # fold_shap_values shape is (8, 512, 20)
    # Average across the outputs/classes (axis 0)
    averaged_shap_values = np.mean(fold_shap_values, axis=0)
    averaged_shap_values_all_folds.append(averaged_shap_values)

# Concatenate and convert to DataFrame
concatenated_averaged_shap_values = np.concatenate(averaged_shap_values_all_folds, axis=0)
averaged_shap_values_df = pd.DataFrame(concatenated_averaged_shap_values)

# 'averaged_shap_values_df' now contains the averaged SHAP values for all samples across all folds


In [ ]:
averaged_shap_values_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.001015,0.001499,-0.031677,0.005340,-0.005632,-0.020547,-0.048760,0.019574,0.014512,0.031816,-0.001767,0.005286,-0.001798,-0.011941,-0.002266,-0.010339,-0.024124,0.028901,-0.006093,0.005533
1,-0.051731,0.006652,0.136839,-0.003894,0.010757,-0.013287,0.037974,-0.017151,-0.022757,-0.015226,-0.001958,0.002843,0.023895,0.024760,-0.015642,-0.002737,-0.016819,0.029769,-0.093086,0.008109
2,0.006235,0.002726,-0.105235,0.002718,0.010556,-0.051326,0.037556,0.037840,0.008996,-0.017308,0.002658,0.004721,-0.010183,0.022277,0.013243,-0.000137,-0.026663,-0.020180,-0.122443,0.004120
3,0.000719,0.005442,0.030199,-0.002578,0.009095,0.080181,0.037520,-0.032054,0.009461,-0.016316,-0.000577,-0.003060,0.000590,0.025039,-0.015776,-0.006022,0.004588,0.029983,-0.104112,-0.002858
4,0.003333,0.006524,-0.005861,-0.000665,0.010431,0.058351,0.037982,-0.014036,0.007418,-0.015662,-0.002117,0.002645,0.001849,0.028881,-0.015081,-0.004178,-0.010822,0.031147,-0.126427,0.003352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,-0.026646,0.001125,0.055896,0.001351,0.026589,-0.209201,0.066349,0.026715,0.008688,0.020754,-0.003117,-0.003833,0.000006,0.017209,0.006311,-0.061827,-0.016674,-0.029071,0.289541,-0.021307
508,-0.041878,0.022439,0.035915,-0.001384,0.026911,0.463933,-0.030851,-0.028827,-0.005917,-0.006023,0.000786,-0.001256,-0.000033,-0.042910,0.009608,-0.012760,-0.027474,-0.030275,-0.072761,-0.021650
509,-0.010746,-0.003816,0.023448,0.001317,0.002640,-0.183422,0.097341,0.033691,0.007331,0.023418,-0.001231,-0.008805,0.000012,0.055271,0.005284,-0.060763,-0.021611,-0.032990,0.267477,-0.010844
510,-0.113499,0.025413,-0.255962,-0.002498,-0.016057,0.508644,-0.026986,-0.080468,0.008861,-0.006110,-0.001233,0.008284,-0.000049,-0.009533,0.007906,-0.046510,0.662046,-0.027435,-0.079064,0.056893


In [87]:
# plot 
shap.summary_plot(averaged_shap_values_df.values[:128], genomic.numpy() if isinstance(genomic, torch.Tensor) else genomic, show=True)
plt.savefig(f'shap_summary_plot_{config_name}.png')

No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


In [46]:


# Train the final model.
models, optimizers = create_models_and_optimizers(config)
lit_model = LitFullModel(models, optimizers, config)
trainer = pl.Trainer(
    default_root_dir=log_path,
    max_epochs=config['max_epochs'],
    enable_progress_bar=False,
    log_every_n_steps=1,
    logger=True,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:

#plot the shap values
shap.summary_plot(all_shap_values_folds, genomic, plot_type="bar")
shap.summary_plot(all_shap_values_folds, genomic)
    

In [47]:
 # get SHAP values from trained model   
feat_extractor = models['feat_ext']
genomic_feat = feat_extractor.genomic_feature_extractor
explanation = shap.DeepExplainer(genomic_feat, genomic)
shap_values = explanation.shap_values(genomic)
print(shap_values)
# plot the SHAP values
shap.summary_plot(shap_values, genomic.numpy()) #, feature_names = feature_ids )  # Convert to numpy if 'genomic' is a tensor
#rename features


plt.savefig('shap_values.png')

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


[array([[-1.66985109e-01,  2.14320087e+00,  9.78210449e+01, ...,
        -2.82952385e+01,  4.27113152e+00, -1.73556018e+00],
       [-4.45447803e-01,  2.16152802e-01,  2.76926346e+01, ...,
         6.42128658e+00, -1.75570316e+01, -1.45657551e+00],
       [ 6.53261662e+00, -8.16797674e-01, -4.15763168e+01, ...,
         6.38731670e+00,  1.19319239e+01, -1.02256620e+00],
       ...,
       [ 2.76738548e+00,  1.65761504e-02, -2.20396099e+01, ...,
         6.13573408e+00,  1.44764309e+01, -1.91254008e+00],
       [-1.77355364e-01,  1.81312785e-01, -3.22547791e+02, ...,
         5.80760574e+00,  1.42741375e+01, -2.34771162e-01],
       [-4.44778830e-01, -3.82962137e-01,  4.21823311e+01, ...,
         5.67802954e+00, -2.69258366e+01,  5.79677820e-01]]), array([[-9.49228883e-01,  9.17250729e+00,  2.22483902e+01, ...,
         2.05281799e+02,  3.33620529e+01, -1.28452158e+00],
       [-2.53215432e+00,  9.25094783e-01,  6.29840231e+00, ...,
        -4.65863838e+01, -1.37138992e+02, -1.07804000

In [20]:

# get SHAP values for all the training data, going through all the batches
all_shap_values = []
for batch in train:
    (genomic, clinical, index, project_id), (overall_survival, survival_time, vital_status) = batch
    shap_values = explanation.shap_values(genomic)
    all_shap_values.append(np.array(shap_values))

all_shap_values = np.concatenate(all_shap_values, axis=1)
all_shap_values
# shap_values_df = pd.DataFrame(all_shap_values)

# shap_values_df.head()

array([[[ 1.35154796e+00, -4.98240113e-01,  3.16914101e+01, ...,
          1.56756096e+01,  1.89143097e+02, -4.53237724e+01],
        [-3.74804664e+00, -3.00861716e+00,  1.41481613e+02, ...,
         -8.01826019e+01, -5.34535179e+01,  1.00351274e-01],
        [-6.98538303e+00, -2.93630719e+00,  1.49345993e+02, ...,
         -8.62959290e+00, -6.00711594e+01,  2.09687042e+00],
        ...,
        [ 1.27547228e+00,  1.35158479e+00, -6.35802879e+01, ...,
          1.37708797e+01, -6.81951981e+01, -2.23266315e+01],
        [-1.87641883e+00, -2.47394371e+00,  5.46146536e+00, ...,
          1.24087152e+01,  1.66692993e+02, -4.75309086e+00],
        [-1.17481737e+01,  3.50825310e-01, -1.03076248e+01, ...,
          1.62941113e+01,  1.48519348e+02, -7.82805395e+00]],

       [[ 2.71542817e-01, -2.71075159e-01, -1.61153774e+01, ...,
          2.32843132e+01,  6.70378799e+01, -2.96517301e+00],
        [-7.53029287e-01, -1.63688612e+00, -7.19447632e+01, ...,
         -1.19102013e+02, -1.89454975e

In [21]:

correlation_matrix = all_shap_values.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation matrix of SHAP values')
plt.savefig('SHAPcorrelation_matrix.png')
        

AttributeError: 'numpy.ndarray' object has no attribute 'corr'